In [ ]:
import sys
sys.path.append('../build')

import ase.io
import numpy as np

import rascal
from rascal.representations import PairDistances

# Testing the calculator on some reference structures

In [ ]:
samples = ase.io.read('../reference_data/inputs/methane_dimer_sample.xyz', ':')

In [ ]:
samples[0].get_distance(0, 5)

First we need to set the unit cells. These are isolated molecular systems, so we just need to "pad" the unit cell enough that periodic images don't show up within the cutoff we've chosen.

In [ ]:
for dim in samples:
    cell_extents = (np.max(dim.get_positions(), axis=0)
                    - np.min(dim.get_positions(), axis=0)
                    + np.array([30, 30, 30]))
    dim.set_cell(cell_extents)
    dim.set_positions(dim.get_positions() + cell_extents / 2.)

Now initialize the calculator with some fairly standard params.

In [ ]:
pd = PairDistances(interaction_cutoff=12.0, cutoff_smooth_width=0.5, n_species=2)

In [ ]:
rep = pd.transform(samples)

In [ ]:
X = rep.get_features(pd)

This is the list of pair distances for the first molecule (90 pairs in total, since each pair comes up twice).  The C-C pair, with the distance we computed above, shows up as the only entry in the final column, which tells us that's the C-C column.  The other two columns are H-H (you can see the repeated intramolecular distance of 1.77 Å) and C-H (with the 1.08 Å C-H bond length again visible).

In [ ]:
X[:90]

### Inverse-power functionality

In [ ]:
pd = PairDistances(interaction_cutoff=12.0, cutoff_smooth_width=0.5, n_species=2,
                   distance_powers=[-6, -8])

In [ ]:
rep = pd.transform(samples)

In [ ]:
X = rep.get_features(pd)

Now you see the same distances, taken to the specified powers.  Since we specified two exponents, the columns are doubled.

In [ ]:
X[:90]

Let's take a look at just the C-C columns:

In [ ]:
np.sum?

In [ ]:
cc_powers = X[:90,4:]
cc_powers = cc_powers[np.any((cc_powers != 0), axis=1)]

In [ ]:
cc_powers

In [ ]:
samples[0].get_distance(0, 5)**-6

In [ ]:
samples[0].get_distance(0, 5)**-8

which correspond to the first and second numbers in these columns.